In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/jane-street-market-prediction/example_sample_submission.csv
/kaggle/input/jane-street-market-prediction/features.csv
/kaggle/input/jane-street-market-prediction/example_test.csv
/kaggle/input/jane-street-market-prediction/train.csv
/kaggle/input/jane-street-market-prediction/janestreet/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/jane-street-market-prediction/janestreet/__init__.py


In [2]:
import os
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

# create the environment
import janestreet
print('Creating competition environment...', end='')
env = janestreet.make_env()
print('Finished.')

Creating competition environment...Finished.


In [3]:
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')

In [4]:
features = [col for col in list(train.columns) if 'feature' in col]

train = train[train['weight'] != 0]

train['action'] = (train['resp'].values > 0).astype(int)

f_mean = train.mean()
train.fillna(f_mean)

X = train.loc[:, features] # Features
y = train['action'].astype(int) # Target variable
X_train,X_test,y_train,y_test = train_test_split(X, y,test_size=0.3, random_state=55)
del train

In [5]:
from catboost import CatBoostClassifier

clf = CatBoostClassifier(depth=6,
                         iterations=500,
                         l2_leaf_reg=1e-20,
                         leaf_estimation_iterations=10,
                         logging_level='Silent',
                         loss_function='Logloss',
                         random_seed= 42)

In [6]:
clf.fit(X_train, y_train)

In [7]:
rcount = 0
for (test_df, prediction_df) in env.iter_test():
    X_test = test_df.loc[:, features]
    y_preds = clf.predict(X_test)
    prediction_df.action = y_preds
    env.predict(prediction_df)
    rcount += len(test_df.index)